In [19]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType,StringType, FloatType
import pyspark
from pyspark.sql import SparkSession

In [20]:
spark = (
    SparkSession.builder.appName("Read_Parquet_File")
    .config("spark.jars.packages",
            "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,"
            "software.amazon.awssdk:bundle:2.24.8,"
            "software.amazon.awssdk:url-connection-client:2.24.8,"
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.375")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.lakehouse_prod_catalog", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.lakehouse_prod_catalog.type", "nessie")  # <--- use nessie, not rest
    .config("spark.sql.catalog.lakehouse_prod_catalog.uri", "http://nessie:19120/api/v2")  # <--- correct URL
    .config("spark.sql.catalog.lakehouse_prod_catalog.warehouse", "s3://warehouse/wh/")
    .config("spark.sql.catalog.lakehouse_prod_catalog.ref", "main")  # <--- default Nessie branch
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")   # MinIO endpoint
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")            # MinIO access key
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")            # MinIO secret key
    .config("spark.hadoop.fs.s3a.path.style.access", "true")           # important for MinIO
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    .getOrCreate()
)

25/10/11 11:11:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [1]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 4.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [12]:
import boto3
import pandas as pd
from io import StringIO

# S3 Credential
aws_access_key_id = "admin"
aws_secret_access_key = "password"

# Create S3 Client for MinIO
s3 = boto3.client(
    "s3",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    endpoint_url="http://minio:9000",  # MinIO endpoint
    use_ssl=False
)

# Bucket Name
bucket_name = "landing-data"

# List objects in bucket
response = s3.list_objects_v2(Bucket=bucket_name)
objects_list = response.get("Contents", [])

# Loop through objects and read CSV into Pandas DataFrame
dfs = []  # list to store DataFrames
for obj in objects_list:
    obj_name = obj["Key"]
    response = s3.get_object(Bucket=bucket_name, Key=obj_name)
    object_content = response["Body"].read().decode("utf-8")

    # Convert CSV text to Pandas DataFrame
    df = pd.read_csv(StringIO(object_content))
    dfs.append(df)

# Combine all files into a single DataFrame (optional)
final_df = pd.concat(dfs, ignore_index=True)

final_df.head()

,Unnamed: 0,FirstName,LastName,StartDate,ExitDate,Title,Supervisor,ADEmail,BusinessUnit,EmployeeStatus,...,Satisfaction Score,Work-Life Balance Score,Training Date,Training Program Name,Training Type,Training Outcome,Location,Trainer,Training Duration(Days),Training Cost
0,0,Uriah,Bridges,20-Sep-19,NaN,Production Technician I,Peter Oneill,uriah.bridges@bilearner.com,CCDR,Active,...,2,3,15-Jul-23,Leadership Development,Internal,Failed,South Marisa,Taylor Rodriguez,2,606.11
1,1,Paula,Small,11-Feb-23,NaN,Production Technician I,Renee Mccormick,paula.small@bilearner.com,EW,Active,...,1,5,12-Sep-22,Customer Service,External,Incomplete,Tammieville,Kelly Patterson DDS,4,673.02
2,2,Edward,Buck,10-Dec-18,NaN,Area Sales Manager,Crystal Walker,edward.buck@bilearner.com,PL,Active,...,2,1,13-Aug-22,Leadership Development,External,Failed,East Roberthaven,Taylor Thomas,2,413.28
3,3,Michael,Riordan,21-Jun-21,NaN,Area Sales Manager,Rebekah Wright,michael.riordan@bilearner.com,CCDR,Active,...,5,4,15-Dec-22,Project Management,External,Completed,Garzatown,Holly Elliott,3,663.78
4,4,Jasmine,Onque,29-Jun-19,NaN,Area Sales Manager,Jason Kim,jasmine.onque@bilearner.com,TNS,Active,...,5,3,13-Jul-23,Technical Skills,External,Failed,Lake Meganville,Donald Martinez,5,399.03


In [21]:
import boto3
import pandas as pd
from io import StringIO

aws_access_key_id = "admin"
aws_secret_access_key = "password"

bucket = "landing-data"
folder = "HR-LAKE-DATASET"
file = "HRIS_EMPLOYEE_20250810.csv"

key = f"{folder}/{file}"

s3 = boto3.client(
    "s3",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    endpoint_url="http://minio:9000",
    use_ssl=False
)

response = s3.get_object(Bucket=bucket, Key=key)
data = response["Body"].read().decode("utf-8")

df = pd.read_csv(StringIO(data))
df.head()


,Unnamed: 0,FirstName,LastName,StartDate,ExitDate,Title,Supervisor,ADEmail,BusinessUnit,EmployeeStatus,...,Satisfaction Score,Work-Life Balance Score,Training Date,Training Program Name,Training Type,Training Outcome,Location,Trainer,Training Duration(Days),Training Cost
0,0,Uriah,Bridges,20-Sep-19,NaN,Production Technician I,Peter Oneill,uriah.bridges@bilearner.com,CCDR,Active,...,2,3,15-Jul-23,Leadership Development,Internal,Failed,South Marisa,Taylor Rodriguez,2,606.11
1,1,Paula,Small,11-Feb-23,NaN,Production Technician I,Renee Mccormick,paula.small@bilearner.com,EW,Active,...,1,5,12-Sep-22,Customer Service,External,Incomplete,Tammieville,Kelly Patterson DDS,4,673.02
2,2,Edward,Buck,10-Dec-18,NaN,Area Sales Manager,Crystal Walker,edward.buck@bilearner.com,PL,Active,...,2,1,13-Aug-22,Leadership Development,External,Failed,East Roberthaven,Taylor Thomas,2,413.28
3,3,Michael,Riordan,21-Jun-21,NaN,Area Sales Manager,Rebekah Wright,michael.riordan@bilearner.com,CCDR,Active,...,5,4,15-Dec-22,Project Management,External,Completed,Garzatown,Holly Elliott,3,663.78
4,4,Jasmine,Onque,29-Jun-19,NaN,Area Sales Manager,Jason Kim,jasmine.onque@bilearner.com,TNS,Active,...,5,3,13-Jul-23,Technical Skills,External,Failed,Lake Meganville,Donald Martinez,5,399.03


In [18]:
spark.stop()